In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
train_image = drive.CreateFile({'id' : '18keD3rntMU1ErXYXYR6jKTe2VXZPa0VU'})
train_image.GetContentFile('train_pos_full.txt')

In [0]:
lines = [line.rstrip('\n') for line in open('train_pos_full.txt')]  #Eureka

In [10]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Unzipping corpora/stopwords.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [0]:
lines1 = []
bwords = stopwords.words('english')

In [0]:

for line in lines :
  sum = ''
  for w in line.split() :
    if w not in bwords :
      sum = sum + w + ' '
  lines1.append(sum.strip())

In [0]:
token = Tokenizer()

In [0]:
token.fit_on_texts(lines1)

In [50]:
len(token.word_index)

263816

In [0]:
lines2 = lines1

In [0]:
seq = token.texts_to_sequences(lines2)

In [0]:
max = 0
for i in seq :
  if len(i) > max :
    max = len(i)

In [0]:
seq = pad_sequences(seq,maxlen = 70,padding = 'post', value = 0)

In [68]:
seq.shape

(1250000, 70)